## CODE INFORMATION:

This code will order all the trains routes (same start and end cities) between 2021 and 2022 by net delay and delay per mile, given that all the necessary train data has already been downloaded.

In [18]:
import requests
from matplotlib import pyplot as plt
import numpy as np
import os
import math

# Train List

In [19]:
# Set and fix the dictionary

# All trains with a sample size of <= 105 from Jan 1 2021 to Dec 31 2022 have had their destinations checked and verified

# Bugged Number Fixes
# 1297 Origin changed to NYP, Destination changed to ALB
# 27 Origin changed to SPK
# 448 Origin changed to CHI
# 550 Origin changed to OAC, Destination changed to SAC

# Changed Destinations to most common destination
# 1105 Destination changed to WPR
# 1106 Destination changed to DEN
# 146 Destination changed to NYP
# 1567 Destination chcnged to OSD
# 167 Destination changed to NYP
# 1765 Destination changed to OSD
# 1777 Destination changed to OSD
# 1785 Destination changed to OSD
# 350 Destination changed to PNT
# 525 Destination changed to OKJ
# 536 Destination changed to SAC
# 711 Destination changed to OKJ
# 742 Destination changed to SAC
# 821 Destination changes to FTW

# Unchanged Destinations Despite Being Uncommon
# 152: Often would go to BOS near end of timeframe. Still sometimes only goes to NYP
# 153: Always would go to NFK near end of timeframe, switched in July 2022
# 169: Often would go to WAS near end of timeframe. Still sometimes only goes to NYP
# 154: Always would go to BOS near end of timeframe, switched in May 2022
# 1763 stopped at GTA 5 times and OSD 6 times. GTA is more recent so we go with that. 
# 184: Always would go to BOS near end of timeframe, switched in April 2022
# 185: Always would go to NFK near end of timeframe, switched in July 2022
# 529 did not operate between April and September 2022, then changed destination from OKJ to SJC.
# 662 is about half NYP, half PHL, so keep NYP as final Destination
# 674: Always would go to NYP near end of timeframe, switched in October 2022

# Changed Origins to most common origin
# 1105 Origin changed to DEN
# 1106 Origin changed to WPR
# 1774 Origin changed to OSD
# 1784 Origin changed to OSD
# 822 Origin changed to FTW

trainDict = {
  1 : [ "NOL" , "LAX" ],
  1003 : [ "CHI" , "LAX" ],
  1004 : [ "LAX" , "CHI" ],
  1005 : [ "CHI" , "EMY" ],
  1006 : [ "EMY" , "CHI" ],
  1007 : [ "CHI" , "SEA" ],
  1008 : [ "SEA" , "CHI" ],
  1011 : [ "SAC" , "LAX" ],
  1014 : [ "KFS" , "SEA" ],
  1027 : [ "SPK" , "PDX" ],
  1028 : [ "PDX" , "CHI" ],
  1058 : [ "NOL" , "CHI" ],
  1059 : [ "CHI" , "NOL" ],
  11 : [ "SEA" , "LAX" ],
  1105 : [ "DEN" , "WPR" ],
  1106 : [ "WPR" , "DEN" ],
  111 : [ "NYP" , "WAS" ],
  1135 : [ "BOS" , "WAS" ],
  1158 : [ "NOL" , "CHI" ],
  1164 : [ "WAS" , "BOS" ],
  1173 : [ "BOS" , "WAS" ],
  1186 : [ "WAS" , "BOS" ],
  1195 : [ "NYP" , "WAS" ],
  121 : [ "NYP" , "WAS" ],
  122 : [ "WAS" , "NYP" ],
  123 : [ "NYP" , "WAS" ],
  1234 : [ "PIT" , "ALB" ],
  1235 : [ "NYP" , "PIT" ],
  124 : [ "NPN" , "NYP" ],
  1244 : [ "PIT" , "NYP" ],
  1245 : [ "ALB" , "PIT" ],
  125 : [ "NYP" , "NPN" ],
  126 : [ "WAS" , "NYP" ],
  129 : [ "NYP" , "WAS" ],
  1297 : [ "NYP" , "ALB" ],
  130 : [ "WAS" , "NYP" ],
  132 : [ "WAS" , "BOS" ],
  134 : [ "WAS" , "NYP" ],
  135 : [ "BOS" , "WAS" ],
  136 : [ "WAS" , "SPG" ],
  137 : [ "BOS" , "WAS" ],
  138 : [ "NFK" , "NYP" ],
  139 : [ "BOS" , "WAS" ],
  14 : [ "LAX" , "SEA" ],
  140 : [ "WAS" , "SPG" ],
  141 : [ "SPG" , "WAS" ],
  143 : [ "SPG" , "WAS" ],
  145 : [ "NYP" , "RNK" ],
  146 : [ "WAS" , "NYP" ],
  147 : [ "SPG" , "RNK" ],
  148 : [ "WAS" , "SPG" ],
  149 : [ "BOS" , "WAS" ],
  150 : [ "NYP" , "BOS" ],
  151 : [ "NYP" , "RNK" ],
  152 : [ "WAS" , "BOS" ],
  153 : [ "NYP" , "NFK" ],
  154 : [ "WAS" , "BOS" ],
  155 : [ "NYP" , "WAS" ],
  156 : [ "RNK" , "NYP" ],
  1562 : [ "OSD" , "SAN" ],
  1564 : [ "LAX" , "SAN" ],
  1567 : [ "SAN" , "OSD" ],
  157 : [ "SPG" , "NFK" ],
  1572 : [ "LAX" , "SAN" ],
  1579 : [ "SAN" , "OSD" ],
  158 : [ "NFK" , "NYP" ],
  1584 : [ "LAX" , "SAN" ],
  159 : [ "NYP" , "WAS" ],
  1590 : [ "OSD" , "SAN" ],
  160 : [ "NYP" , "BOS" ],
  161 : [ "BOS" , "WAS" ],
  162 : [ "WAS" , "BOS" ],
  163 : [ "BOS" , "WAS" ],
  164 : [ "RVM" , "BOS" ],
  165 : [ "BOS" , "WAS" ],
  166 : [ "WAS" , "BOS" ],
  167 : [ "BOS" , "NYP" ],
  168 : [ "WAS" , "BOS" ],
  1689 : [ "BON" , "BRK" ],
  169 : [ "BOS" , "WAS" ],
  170 : [ "WAS" , "BOS" ],
  171 : [ "BOS" , "RNK" ],
  172 : [ "WAS" , "BOS" ],
  173 : [ "BOS" , "WAS" ],
  174 : [ "NPN" , "BOS" ],
  175 : [ "BOS" , "WAS" ],
  176 : [ "RNK" , "BOS" ],
  1761 : [ "SAN" , "SLO" ],
  1763 : [ "SAN" , "GTA" ],
  1765 : [ "SAN" , "OSD" ],
  1767 : [ "SAN" , "LAX" ],
  1768 : [ "GTA" , "SAN" ],
  177 : [ "BOS" , "WAS" ],
  1770 : [ "GTA" , "SAN" ],
  1774 : [ "OSD" , "SAN" ],
  1777 : [ "SAN" , "OSD" ],
  178 : [ "WAS" , "BOS" ],
  1784 : [ "OSD" , "SAN" ],
  1785 : [ "SAN" , "OSD" ],
  179 : [ "BOS" , "NYP" ],
  1790 : [ "OSD" , "SAN" ],
  1793 : [ "SAN" , "OSD" ],
  1794 : [ "SLO" , "OXN" ],
  1796 : [ "GTA" , "OXN" ],
  180 : [ "WAS" , "NYP" ],
  181 : [ "NYP" , "WAS" ],
  182 : [ "WAS" , "NYP" ],
  183 : [ "NYP" , "WAS" ],
  184 : [ "MET" , "BOS" ],
  185 : [ "NYP" , "NFK" ],
  186 : [ "NPN" , "NYP" ],
  189 : [ "NYP" , "WAS" ],
  19 : [ "NYP" , "NOL" ],
  190 : [ "NYP" , "BOS" ],
  192 : [ "WAS" , "NYP" ],
  193 : [ "NYP" , "WAS" ],
  194 : [ "NPN" , "BOS" ],
  195 : [ "BOS" , "RVM" ],
  196 : [ "WAS" , "NYP" ],
  198 : [ "WAS" , "NYP" ],
  2 : [ "LAX" , "NOL" ],
  20 : [ "NOL" , "NYP" ],
  21 : [ "CHI" , "SAS" ],
  2103 : [ "NYP" , "WAS" ],
  2106 : [ "WAS" , "NYP" ],
  2107 : [ "NYP" , "WAS" ],
  2109 : [ "NYP" , "WAS" ],
  2121 : [ "NYP" , "WAS" ],
  2122 : [ "WAS" , "NYP" ],
  2126 : [ "WAS" , "NYP" ],
  2128 : [ "WAS" , "NYP" ],
  2150 : [ "WAS" , "BOS" ],
  2151 : [ "BOS" , "WAS" ],
  2152 : [ "WAS" , "BOS" ],
  2153 : [ "BOS" , "WAS" ],
  2154 : [ "WAS" , "BOS" ],
  2155 : [ "BOS" , "WAS" ],
  2156 : [ "WAS" , "BOS" ],
  2158 : [ "WAS" , "BOS" ],
  2159 : [ "BOS" , "WAS" ],
  2160 : [ "WAS" , "BOS" ],
  2163 : [ "BOS" , "WAS" ],
  2164 : [ "WAS" , "BOS" ],
  2165 : [ "BOS" , "WAS" ],
  2166 : [ "WAS" , "BOS" ],
  2167 : [ "BOS" , "WAS" ],
  2168 : [ "WAS" , "BOS" ],
  2169 : [ "BOS" , "WAS" ],
  2170 : [ "WAS" , "BOS" ],
  2172 : [ "WAS" , "BOS" ],
  2173 : [ "BOS" , "WAS" ],
  2175 : [ "BOS" , "WAS" ],
  2190 : [ "NYP" , "BOS" ],
  2193 : [ "BOS" , "NYP" ],
  22 : [ "SAS" , "CHI" ],
  2203 : [ "NYP" , "WAS" ],
  2205 : [ "NYP" , "WAS" ],
  2213 : [ "NYP" , "WAS" ],
  2217 : [ "NYP" , "WAS" ],
  2218 : [ "WAS" , "NYP" ],
  2220 : [ "WAS" , "NYP" ],
  2222 : [ "WAS" , "NYP" ],
  2224 : [ "WAS" , "NYP" ],
  2226 : [ "WAS" , "NYP" ],
  2233 : [ "BOS" , "WAS" ],
  2235 : [ "BOS" , "WAS" ],
  2239 : [ "BOS" , "WAS" ],
  2248 : [ "WAS" , "BOS" ],
  2249 : [ "BOS" , "WAS" ],
  2250 : [ "WAS" , "BOS" ],
  2251 : [ "BOS" , "WAS" ],
  2252 : [ "WAS" , "BOS" ],
  2253 : [ "BOS" , "WAS" ],
  2254 : [ "WAS" , "BOS" ],
  2255 : [ "BOS" , "WAS" ],
  2256 : [ "WAS" , "BOS" ],
  2257 : [ "BOS" , "WAS" ],
  2259 : [ "BOS" , "WAS" ],
  2260 : [ "WAS" , "BOS" ],
  2275 : [ "BOS" , "WAS" ],
  2290 : [ "NYP" , "BOS" ],
  2292 : [ "NYP" , "BOS" ],
  230 : [ "ALB" , "NYP" ],
  232 : [ "ALB" , "NYP" ],
  233 : [ "NYP" , "ALB" ],
  234 : [ "ALB" , "NYP" ],
  235 : [ "NYP" , "ALB" ],
  236 : [ "ALB" , "NYP" ],
  237 : [ "NYP" , "ALB" ],
  238 : [ "ALB" , "NYP" ],
  239 : [ "NYP" , "ALB" ],
  240 : [ "ALB" , "NYP" ],
  241 : [ "NYP" , "ALB" ],
  243 : [ "NYP" , "ALB" ],
  244 : [ "ALB" , "NYP" ],
  250 : [ "ALB" , "NYP" ],
  253 : [ "NYP" , "ALB" ],
  255 : [ "NYP" , "ALB" ],
  256 : [ "ALB" , "NYP" ],
  259 : [ "NYP" , "ALB" ],
  260 : [ "ALB" , "NYP" ],
  261 : [ "NYP" , "ALB" ],
  27 : [ "SPK" , "PDX" ],
  28 : [ "PDX" , "SPK" ],
  280 : [ "NFL" , "NYP" ],
  281 : [ "NYP" , "NFL" ],
  283 : [ "NYP" , "NFL" ],
  284 : [ "NFL" , "NYP" ],
  29 : [ "WAS" , "CHI" ],
  290 : [ "RUD" , "NYP" ],
  291 : [ "NYP" , "BTN" ],
  292 : [ "RUD" , "NYP" ],
  293 : [ "NYP" , "RUD" ],
  295 : [ "NYP" , "RUD" ],
  296 : [ "RUD" , "NYP" ],
  3 : [ "CHI" , "LAX" ],
  30 : [ "CHI" , "WAS" ],
  300 : [ "STL" , "CHI" ],
  301 : [ "CHI" , "STL" ],
  302 : [ "STL" , "CHI" ],
  303 : [ "CHI" , "STL" ],
  304 : [ "STL" , "CHI" ],
  305 : [ "CHI" , "STL" ],
  306 : [ "STL" , "CHI" ],
  307 : [ "CHI" , "STL" ],
  311 : [ "STL" , "KCY" ],
  313 : [ "STL" , "KCY" ],
  314 : [ "KCY" , "STL" ],
  316 : [ "KCY" , "STL" ],
  318 : [ "KCY" , "CHI" ],
  319 : [ "CHI" , "KCY" ],
  329 : [ "CHI" , "MKE" ],
  330 : [ "MKE" , "CHI" ],
  331 : [ "CHI" , "MKE" ],
  332 : [ "MKE" , "CHI" ],
  333 : [ "CHI" , "MKE" ],
  334 : [ "MKE" , "CHI" ],
  335 : [ "CHI" , "MKE" ],
  336 : [ "MKE" , "CHI" ],
  337 : [ "CHI" , "MKE" ],
  338 : [ "MKE" , "CHI" ],
  339 : [ "CHI" , "MKE" ],
  340 : [ "MKE" , "CHI" ],
  341 : [ "CHI" , "MKE" ],
  342 : [ "MKE" , "CHI" ],
  343 : [ "CHI" , "MKE" ],
  350 : [ "CHI" , "PNT" ],
  351 : [ "PNT" , "CHI" ],
  352 : [ "CHI" , "PNT" ],
  353 : [ "PNT" , "CHI" ],
  354 : [ "CHI" , "PNT" ],
  355 : [ "PNT" , "CHI" ],
  364 : [ "CHI" , "PTH" ],
  365 : [ "PTH" , "CHI" ],
  370 : [ "CHI" , "GRR" ],
  371 : [ "GRR" , "CHI" ],
  380 : [ "QCY" , "CHI" ],
  381 : [ "CHI" , "QCY" ],
  382 : [ "QCY" , "CHI" ],
  383 : [ "CHI" , "QCY" ],
  390 : [ "CDL" , "CHI" ],
  391 : [ "CHI" , "CDL" ],
  392 : [ "CDL" , "CHI" ],
  393 : [ "CHI" , "CDL" ],
  4 : [ "LAX" , "CHI" ],
  400 : [ "SPG" , "GFD" ],
  405 : [ "SPG" , "NHV" ],
  409 : [ "SPG" , "NHV" ],
  412 : [ "NHV" , "SPG" ],
  416 : [ "NHV" , "SPG" ],
  417 : [ "SPG" , "NHV" ],
  42 : [ "PGH" , "NYP" ],
  43 : [ "NYP" , "PGH" ],
  432 : [ "NHV" , "SPG" ],
  448 : [ "CHI" , "BOS" ],
  449 : [ "BOS" , "ALB" ],
  450 : [ "NHV" , "SPG" ],
  451 : [ "SPG" , "NHV" ],
  458 : [ "NHV" , "SPG" ],
  460 : [ "NHV" , "SPG" ],
  461 : [ "GFD" , "NHV" ],
  463 : [ "SPG" , "NHV" ],
  464 : [ "NHV" , "SPG" ],
  465 : [ "SPG" , "NHV" ],
  467 : [ "SPG" , "NHV" ],
  470 : [ "NHV" , "SPG" ],
  471 : [ "GFD" , "NHV" ],
  473 : [ "SPG" , "NHV" ],
  474 : [ "NHV" , "SPG" ],
  475 : [ "SPG" , "NHV" ],
  476 : [ "NHV" , "SPG" ],
  478 : [ "NHV" , "GFD" ],
  479 : [ "SPG" , "NHV" ],
  48 : [ "CHI" , "NYP" ],
  488 : [ "NHV" , "GFD" ],
  49 : [ "NYP" , "CHI" ],
  490 : [ "NHV" , "SPG" ],
  494 : [ "NHV" , "GFD" ],
  495 : [ "GFD" , "NHV" ],
  497 : [ "SPG" , "NHV" ],
  499 : [ "GFD" , "SPG" ],
  5 : [ "CHI" , "EMY" ],
  50 : [ "CHI" , "NYP" ],
  500 : [ "EUG" , "SEA" ],
  502 : [ "PDX" , "SEA" ],
  503 : [ "SEA" , "EUG" ],
  504 : [ "PDX" , "SEA" ],
  505 : [ "SEA" , "EUG" ],
  507 : [ "SEA" , "PDX" ],
  508 : [ "EUG" , "SEA" ],
  509 : [ "SEA" , "PDX" ],
  51 : [ "NYP" , "CHI" ],
  516 : [ "SEA" , "VAC" ],
  519 : [ "VAC" , "SEA" ],
  52 : [ "SFA" , "LOR" ],
  520 : [ "OKJ" , "SAC" ],
  521 : [ "SAC" , "SJC" ],
  522 : [ "OKJ" , "SAC" ],
  523 : [ "SAC" , "SJC" ],
  524 : [ "SJC" , "SAC" ],
  525 : [ "SAC" , "OKJ" ],
  526 : [ "OKJ" , "SAC" ],
  527 : [ "ARN" , "SJC" ],
  528 : [ "SJC" , "SAC" ],
  529 : [ "ARN" , "SJC" ],
  53 : [ "LOR" , "SFA" ],
  531 : [ "SAC" , "SJC" ],
  532 : [ "SJC" , "SAC" ],
  533 : [ "OKJ" , "SJC" ],
  534 : [ "OKJ" , "SAC" ],
  535 : [ "SAC" , "OKJ" ],
  536 : [ "OKJ" , "SAC" ],
  537 : [ "SAC" , "SJC" ],
  538 : [ "SJC" , "ARN" ],
  54 : [ "WAS" , "SAB" ],
  540 : [ "OKJ" , "SAC" ],
  541 : [ "SAC" , "SJC" ],
  542 : [ "SJC" , "SAC" ],
  543 : [ "SAC" , "OKJ" ],
  544 : [ "SJC" , "SAC" ],
  545 : [ "SAC" , "OKJ" ],
  546 : [ "SJC" , "SAC" ],
  547 : [ "SAC" , "SJC" ],
  548 : [ "OKJ" , "SAC" ],
  549 : [ "SAC" , "OKJ" ],
  55 : [ "SAB" , "WAS" ],
  550 : [ "OAC" , "SAC" ],
  551 : [ "SAC" , "OKJ" ],
  552 : [ "SJC" , "SAC" ],
  56 : [ "WAS" , "SAB" ],
  562 : [ "LAX" , "SAN" ],
  564 : [ "LAX" , "SAN" ],
  567 : [ "SAN" , "LAX" ],
  568 : [ "LAX" , "SAN" ],
  569 : [ "SAN" , "LAX" ],
  57 : [ "SAB" , "WAS" ],
  572 : [ "LAX" , "SAN" ],
  573 : [ "SAN" , "LAX" ],
  579 : [ "SAN" , "LAX" ],
  58 : [ "NOL" , "CHI" ],
  580 : [ "LAX" , "SAN" ],
  581 : [ "SAN" , "LAX" ],
  583 : [ "SAN" , "LAX" ],
  584 : [ "LAX" , "SAN" ],
  586 : [ "LAX" , "SAN" ],
  588 : [ "LAX" , "SAN" ],
  589 : [ "SAN" , "LAX" ],
  59 : [ "CHI" , "NOL" ],
  590 : [ "LAX" , "SAN" ],
  591 : [ "SAN" , "LAX" ],
  593 : [ "SAN" , "LAX" ],
  594 : [ "LAX" , "SAN" ],
  595 : [ "SAN" , "LAX" ],
  6 : [ "EMY" , "CHI" ],
  600 : [ "HAR" , "NYP" ],
  601 : [ "PHL" , "HAR" ],
  605 : [ "PHL" , "HAR" ],
  607 : [ "PHL" , "HAR" ],
  609 : [ "NYP" , "HAR" ],
  610 : [ "HAR" , "PHL" ],
  611 : [ "PHL" , "HAR" ],
  612 : [ "HAR" , "PHL" ],
  615 : [ "PHL" , "HAR" ],
  618 : [ "HAR" , "PHL" ],
  619 : [ "PHL" , "HAR" ],
  620 : [ "HAR" , "PHL" ],
  622 : [ "HAR" , "PHL" ],
  624 : [ "HAR" , "NYP" ],
  63 : [ "NYP" , "TWO" ],
  637 : [ "NYP" , "PHL" ],
  638 : [ "PHL" , "NYP" ],
  639 : [ "NYP" , "PHL" ],
  64 : [ "TWO" , "NYP" ],
  640 : [ "HAR" , "NYP" ],
  641 : [ "NYP" , "HAR" ],
  642 : [ "HAR" , "NYP" ],
  643 : [ "NYP" , "HAR" ],
  644 : [ "HAR" , "NYP" ],
  645 : [ "NYP" , "HAR" ],
  646 : [ "HAR" , "NYP" ],
  647 : [ "NYP" , "HAR" ],
  648 : [ "HAR" , "NYP" ],
  649 : [ "NYP" , "HAR" ],
  65 : [ "BOS" , "NPN" ],
  650 : [ "HAR" , "NYP" ],
  651 : [ "NYP" , "HAR" ],
  652 : [ "HAR" , "NYP" ],
  653 : [ "NYP" , "HAR" ],
  654 : [ "HAR" , "NYP" ],
  655 : [ "NYP" , "HAR" ],
  656 : [ "HAR" , "NYP" ],
  657 : [ "NYP" , "HAR" ],
  658 : [ "HAR" , "NYP" ],
  66 : [ "RNK" , "BOS" ],
  660 : [ "HAR" , "NYP" ],
  661 : [ "NYP" , "HAR" ],
  662 : [ "HAR" , "NYP" ],
  663 : [ "NYP" , "HAR" ],
  664 : [ "HAR" , "NYP" ],
  665 : [ "NYP" , "HAR" ],
  666 : [ "HAR" , "NYP" ],
  667 : [ "NYP" , "HAR" ],
  669 : [ "NYP" , "HAR" ],
  67 : [ "BOS" , "NPN" ],
  670 : [ "HAR" , "NYP" ],
  671 : [ "NYP" , "HAR" ],
  672 : [ "HAR" , "NYP" ],
  674 : [ "HAR" , "NYP" ],
  68 : [ "ALB" , "NYP" ],
  680 : [ "BRK" , "BON" ],
  681 : [ "BON" , "BRK" ],
  682 : [ "BRK" , "BON" ],
  683 : [ "BON" , "BRK" ],
  684 : [ "BRK" , "BON" ],
  685 : [ "BON" , "BRK" ],
  686 : [ "BRK" , "BON" ],
  687 : [ "BON" , "BRK" ],
  688 : [ "BRK" , "BON" ],
  689 : [ "BON" , "BRK" ],
  69 : [ "NYP" , "ALB" ],
  690 : [ "BRK" , "BON" ],
  691 : [ "BON" , "BRK" ],
  692 : [ "BRK" , "BON" ],
  693 : [ "BON" , "BRK" ],
  694 : [ "BRK" , "BON" ],
  695 : [ "BON" , "BRK" ],
  696 : [ "BRK" , "BON" ],
  697 : [ "BON" , "BRK" ],
  698 : [ "BRK" , "BON" ],
  699 : [ "BON" , "BRK" ],
  7 : [ "CHI" , "SEA" ],
  702 : [ "SAC" , "BFD" ],
  703 : [ "BFD" , "SAC" ],
  710 : [ "OKJ" , "BFD" ],
  711 : [ "BFD" , "OKJ" ],
  712 : [ "OKJ" , "BFD" ],
  713 : [ "BFD" , "OKJ" ],
  714 : [ "OKJ" , "BFD" ],
  715 : [ "BFD" , "OKJ" ],
  716 : [ "OKJ" , "BFD" ],
  717 : [ "BFD" , "OKJ" ],
  718 : [ "OAC" , "BFD" ],
  719 : [ "BFD" , "OKJ" ],
  720 : [ "OKJ" , "SAC" ],
  723 : [ "SAC" , "SJC" ],
  724 : [ "SJC" , "SAC" ],
  727 : [ "SAC" , "SJC" ],
  728 : [ "SJC" , "SAC" ],
  729 : [ "ARN" , "SJC" ],
  73 : [ "RGH" , "CLT" ],
  732 : [ "SJC" , "SAC" ],
  733 : [ "SAC" , "OAC" ],
  734 : [ "OKJ" , "SAC" ],
  736 : [ "SJC" , "ARN" ],
  737 : [ "SAC" , "SJC" ],
  738 : [ "SJC" , "SAC" ],
  74 : [ "CLT" , "RGH" ],
  741 : [ "SAC" , "SJC" ],
  742 : [ "SJC" , "SAC" ],
  743 : [ "SAC" , "SJC" ],
  744 : [ "SJC" , "SAC" ],
  745 : [ "SAC" , "OKJ" ],
  746 : [ "OKJ" , "SAC" ],
  747 : [ "SAC" , "SJC" ],
  748 : [ "SJC" , "SAC" ],
  749 : [ "SAC" , "OKJ" ],
  75 : [ "RGH" , "CLT" ],
  751 : [ "SAC" , "OKJ" ],
  76 : [ "CLT" , "RGH" ],
  761 : [ "SAN" , "SLO" ],
  763 : [ "SAN" , "GTA" ],
  765 : [ "SAN" , "GTA" ],
  767 : [ "SAN" , "LAX" ],
  768 : [ "GTA" , "SAN" ],
  77 : [ "RGH" , "CLT" ],
  770 : [ "GTA" , "SAN" ],
  774 : [ "SLO" , "SAN" ],
  777 : [ "SAN" , "SLO" ],
  78 : [ "CLT" , "RGH" ],
  784 : [ "GTA" , "SAN" ],
  785 : [ "SAN" , "GTA" ],
  79 : [ "NYP" , "CLT" ],
  794 : [ "SLO" , "LAX" ],
  796 : [ "GTA" , "SAN" ],
  8 : [ "SEA" , "CHI" ],
  80 : [ "CLT" , "NYP" ],
  82 : [ "NFK" , "BOS" ],
  821 : [ "OKC" , "FTW" ],
  822 : [ "FTW" , "OKC" ],
  84 : [ "NFK" , "NYP" ],
  85 : [ "BOS" , "RVM" ],
  86 : [ "RVM" , "BOS" ],
  87 : [ "BOS" , "NFK" ],
  88 : [ "NFK" , "BOS" ],
  89 : [ "NYP" , "SAV" ],
  90 : [ "SAV" , "NYP" ],
  91 : [ "NYP" , "MIA" ],
  92 : [ "MIA" , "NYP" ],
  93 : [ "BOS" , "NFK" ],
  94 : [ "NFK" , "BOS" ],
  95 : [ "BOS" , "NFK" ],
  96 : [ "NPN" , "BOS" ],
  97 : [ "NYP" , "MIA" ],
  98 : [ "MIA" , "NYP" ],
  99 : [ "BOS" , "NPN" ],
}

# List of all Train Routes

In [20]:
# Get list of all the unique origin + destination combos
trainRoutes = { }
for x in trainDict:
    if (trainRoutes.get(trainDict[x][0]) == None):
        trainRoutes[trainDict[x][0]] = []
    trainRoutes[trainDict[x][0]].append(trainDict[x][1])

# Assign Each Train to a Route

In [21]:
# For each origin/destination combo, get the trains that take it
routeTrains = { }
for x in trainRoutes:
    for y in range(len(trainRoutes[x])):
        routeTrains[tuple([x, trainRoutes[x][y]])] = []
        
for x in trainDict:
    routeTrains[tuple([trainDict[x][0], trainDict[x][1]])].append(x)

# Coordinates of Train Stations

In [22]:
stationCoordinates = {
 'ABE': ['Aberdeen, MD', 39.5084786, -76.1631584],
 'ABQ': ['Albuquerque, NM', 35.0823627, -106.6482287],
 'ACA': ['Antioch-Pittsburg, CA', 38.01777850000001, -121.816168],
 'ACD': ['Arcadia, MO', 37.5922969, -90.6243831],
 'ADM': ['Ardmore, OK', 34.1727698, -97.12532449999999],
 'AKY': ['Ashland, KY', 38.4810607, -82.63993839999999],
 'ALB': ['Albany-Rensselaer, NY', 42.6410631, -73.7412044],
 'ALC': ['Alliance, OH', 40.9211684, -81.09249109999999],
 'ALD': ['Alderson, WV', 37.7243179, -80.6448262],
 'ALI': ['Albion, MI', 42.243097, -84.7530304],
 'ALN': ['Alton, IL', 38.9211205, -90.1573597],
 'ALP': ['Alpine, TX', 30.3581727, -103.6618239],
 'ALT': ['Altoona, PA', 40.5186809, -78.3947359],
 'ALX': ['Alexandria, VA', 38.8048355, -77.0469214],
 'ALY': ['Albany, OR', 44.6304372, -123.1026643],
 'AMM': ['Amherst, MA', 42.3752316, -72.5116512],
 'AMS': ['Amsterdam, NY', 42.95368200000001, -74.2195315],
 'ANA': ['Anaheim, CA', 33.8034269, -117.8774559],
 'ARB': ['Ann Arbor, MI', 42.287777, -83.74318459999999],
 'ARD': ['Ardmore, PA', 40.0082965, -75.29029299999999],
 'ARK': ['Arkadelphia, AR', 34.1141726, -93.05296349999999],
 'ARN': ['Auburn, CA', 38.8965654, -121.0768901],
 'ASD': ['Ashland, VA', 37.7596208, -77.4812137],
 'ATL': ['Atlanta, GA', 33.7994274, -84.3925888],
 'ATN': ['Anniston, AL', 33.649112, -85.832145],
 'AUS': ['Austin, TX', 30.2695814, -97.7570741],
 'BAL': ['Baltimore - Penn Station, MD', 39.3073829, -76.6155859],
 'BAM': ['Bangor, MI', 42.3125355, -86.11307939999999],
 'BAR': ['Barstow, CA', 34.9047063, -117.0247091],
 'BBK': ['Burbank - Downtown, CA', 34.1808392, -118.3089661],
 'BBY': ['Boston - Back Bay, MA', 42.3475629, -71.07534369999999],
 'BCV': ['Burke - Burke Centre, VA', 38.7974014, -77.2988039],
 'BEL': ['Bellingham, WA', 48.72042709999999, -122.511129],
 'BEN': ['Benson, AZ', 31.9688086, -110.2970601],
 'BER': ['Berlin, CT', 41.63596039999999, -72.7650057],
 'BFD': ['Bakersfield, CA', 35.3721912, -119.0076905],
 'BFX': ['Buffalo - Exchange St., NY', 42.8783736, -78.8737852],
 'BHG': ['Benton Harbor- Harbor Shores Golf Club, MI', 42.1198447, -86.45941],
 'BHM': ['Birmingham, AL', 33.5126667, -86.80665119999999],
 'BKY': ['Berkeley, CA', 37.8715226, -122.273042],
 'BLF': ['Bellows Falls, VT', 43.133411, -72.44397719999999],
 'BMM': ['Birmingham, MI', 42.5455244, -83.1942495],
 'BMT': ['Beaumont, TX', 30.0763445, -94.1276121],
 'BNC': ['Burlington, NC', 36.0941731, -79.4346552],
 'BNG': ['Bingen-White Salmon, WA', 45.7157629, -121.4687017],
 'BNL': ['Bloomington-Normal, IL', 40.5088799, -88.9843499],
 'BON': ['Boston - North Station Maine Service, MA', 42.3661096, -71.0630893],
 'BOS': ['Boston - South Station, MA', 42.3519217, -71.0550703],
 'BRA': ['Brattleboro, VT', 42.8509152, -72.5578678],
 'BRH': ['Brookhaven, MS', 31.583077, -90.4408892],
 'BRK': ['Brunswick, ME', 43.91157459999999, -69.96551869999999],
 'BRL': ['Burlington, IA', 40.8057592, -91.1017536],
 'BRO': ['Browning, MT', 48.5339813, -113.0134506],
 'BRP': ['Bridgeport, CT', 41.1792258, -73.1894384],
 'BTL': ['Battle Creek, MI', 42.3183345, -85.1876715],
 'BTN': ['Burlington, VT', 44.475723, -73.219538],
 'BUF': ['Buffalo - Depew, NY', 42.9072488, -78.7270496],
 'BUR': ['Burbank - Airport, CA', 34.192895, -118.3534947],
 'BWI': ['Baltimore - BWI Thurgood Marshall Airport, MD', 39.1924679, -76.6943881],
 'BYN': ['Bryan, OH', 41.4803036, -84.551879],
 'CAM': ['Camden, SC', 34.248157, -80.62518999999999],
 'CBN': ['Canadian Border', 56.130366, -106.346771],
 'CBR': ['Cleburne, TX', 32.3497641, -97.3822129],
 'CBS': ['Columbus, WI', 43.3407126, -89.012591],
 'CBV': ['Carlsbad - Village, CA', 33.1609289, -117.3508223],
 'CDL': ['Carbondale, IL', 37.7272727, -89.2167501],
 'CEN': ['Centralia, IL', 38.5274161, -89.13608669999999],
 'CHI': ['Chicago - Union Station, IL', 41.8786897, -87.640311],
 'CHM': ['Champaign-Urbana, IL', 40.1158323, -88.241186],
 'CHS': ['Charleston, SC', 32.875428, -79.99821419999999],
 'CHW': ['Charleston, WV', 38.3463176, -81.63827669999999],
 'CIC': ['Chico, CA', 39.7233735, -121.8461365],
 'CIN': ['Cincinnati, OH', 39.1099204, -84.5376308],
 'CLA': ['Claremont, NH', 43.369134, -72.3776468],
 'CLB': ['Columbia, SC', 33.9942692, -81.0402857],
 'CLE': ['Cleveland, OH', 41.49932, -81.6943605],
 'CLF': ['Clifton Forge, VA', 37.8143672, -79.8281154],
 'CLP': ['Culpeper, VA', 38.4721662, -77.9934703],
 'CLT': ['Charlotte, NC', 35.241285, -80.82270799999999],
 'CML': ['Camarillo, CA', 34.2163937, -119.0376023],
 'CMO': ['Chemult, OR', 43.21667009999999, -121.781431],
 'CNL': [''],
 'CNV': ['Castleton, VT', 43.613456, -73.1713041],
 'COC': ['Corcoran, CA', 36.0983363, -119.5571298],
 'COI': ['Connersville, IN', 39.6459831, -85.1331622],
 'COT': ['Coatesville, PA', 39.9857503, -75.8209438],
 'COV': ['Connellsville, PA', 40.0204268, -79.5927935],
 'COX': ['Colfax, CA', 39.0990124, -120.9532317],
 'CPN': ['Carpinteria, CA', 34.39888380000001, -119.5184564],
 'CRF': ['Crawfordsville, IN', 40.0411536, -86.87445160000001],
 'CRN': ['Creston, IA', 41.0570373, -94.3624947],
 'CRT': ['Croton-Harmon, NY', 41.1898375, -73.88262929999999],
 'CRV': ['Carlinville, IL', 39.2789901, -89.8894113],
 'CSN': ['Clemson, SC', 34.691031, -82.832503],
 'CTL': ['Centralia, WA', 46.71739609999999, -122.952925],
 'CUM': ['Cumberland, MD', 39.6505246, -78.7579373],
 'CUT': ['Cut Bank, MT', 48.638504, -112.331516],
 'CVS': ['Charlottesville, VA', 38.0314574, -78.491811],
 'CWH': ['Cornwells Heights, PA', 40.0717356, -74.9520878],
 'CWT': ['Chatsworth, CA', 34.25301160000001, -118.5994635],
 'CYN': ['Cary, NC', 35.7884141, -78.7822868],
 'DAL': ['Dallas, TX', 32.7762167, -96.80754669999999],
 'DAN': ['Danville, VA', 36.5840605, -79.38381249999999],
 'DAV': ['Davis, CA', 38.5434908, -121.7377498],
 'DDG': ['Dodge City, KS', 37.7524206, -100.0170722],
 'DEM': ['Deming, NM', 32.2716258, -107.7541701],
 'DEN': ['Denver, CO', 39.7533257, -105.0004303],
 'DER': ['Dearborn, MI', 42.3070375, -83.23522200000001],
 'DET': ['Detroit, MI', 42.36806929999999, -83.0722796],
 'DFB': ['Deerfield Beach, FL', 26.3168109, -80.1222133],
 'DHM': ['Durham, NH', 43.139353, -70.9361201],
 'DIL': ['Dillon, SC', 34.4175825, -79.3721855],
 'DLB': ['Delray Beach, FL', 26.4541386, -80.09105869999999],
 'DLD': ['Deland, FL', 29.0178446, -81.3526292],
 'DLK': ['Detroit Lakes, MN', 46.8197994, -95.84598489999999],
 'DNC': ['Durham, NC', 35.9975138, -78.9064757],
 'DNK': ['Denmark, SC', 33.3262468, -81.1436081],
 'DOA': ['Dowagiac, MI', 41.9809945, -86.10889259999999],
 'DOV': ['Dover, NH', 43.1980919, -70.8781888],
 'DOW': ['Downingtown, PA', 40.0021519, -75.71076959999999],
 'DQN': ['Du Quoin, IL', 38.0122464, -89.2401462],
 'DRD': ['Durand, MI', 42.909375, -83.9825218],
 'DRT': ['Del Rio, TX', 29.3623444, -100.9026164],
 'DUN': ['Dunsmuir, CA', 41.2112326, -122.2705861],
 'DVL': ['Devils Lake, ND', 48.1103899, -98.86151939999999],
 'DWT': ['Dwight, IL', 41.0899135, -88.4308085],
 'DYE': ['Dyer, IN', 41.5156247, -87.5181807],
 'EDM': ['Edmonds, WA', 47.81045, -122.38519],
 'EFG': ['Effingham, IL', 39.1200418, -88.5433829],
 'EKH': ['Elkhart, IN', 41.6806628, -85.9717569],
 'ELK': ['Elko, NV', 40.8368386, -115.7503713],
 'ELP': ['El Paso, TX', 31.7575712, -106.4960664],
 'ELT': ['Elizabethtown, PA', 40.1469142, -76.6122271],
 'ELY': ['Elyria, OH', 41.3701348, -82.0965713],
 'EMP': ['Emporia, KS', 38.403903, -96.18166260000001],
 'EMY': ['Emeryville, CA', 37.8404464, -122.2917273],
 'ENC': ['Encinitas, CA', 33.0369867, -117.2919818],
 'EPH': ['Ephrata, WA', 47.32084649999999, -119.5492993],
 'ERI': ['Erie, PA', 42.1207391, -80.0823182],
 'ESM': ['Essex, MT', 48.2780178, -113.6126035],
 'ESX': ['Essex Junction, VT', 44.4925812, -73.1101592],
 'EUG': ['Eugene, OR', 44.0551578, -123.0922546],
 'EVR': ['Everett, WA', 47.9789848, -122.2020794],
 'EWR': ['Newark - International Airport, NJ', 40.70441, -74.19072],
 'EXR': ['Exeter, NH', 42.98102000000001, -70.95870000000001],
 'EXT': ['Exton, PA', 40.0195261, -75.6218097],
 'FAL': ['Falmouth, ME', 43.7435343, -70.2601759],
 'FAR': ['Fargo, ND', 46.8809765, -96.78513509999999],
 'FAY': ['Fayetteville, NC', 35.0550748, -78.8847646],
 'FBG': ['Fredericksburg, VA', 38.2982698, -77.4569358],
 'FED': ['Fort Edward-Glens Falls, NY', 43.2697423, -73.58039550000001],
 'FFV': ['Fairfield-Vacaville, CA', 38.2853569, -121.9681155],
 'FHV': ['Fair Haven, VT', 43.594801, -73.2657155],
 'FLG': ['Flagstaff, AZ', 35.1973203, -111.6491951],
 'FLN': ['Flint, MI', 43.015445, -83.6515373],
 'FLO': ['Florence, SC', 34.1993038, -79.7575219],
 'FMD': ['Fort Madison, IA', 40.6295325, -91.313644],
 'FMG': ['Fort Morgan, CO', 40.2461801, -103.8031269],
 'FMT': ['Fremont, CA', 37.5591357, -122.007382],
 'FNO': ['Fresno, CA', 36.7384016, -119.7826731],
 'FRA': ['Framingham, MA', 42.279286, -71.4161565],
 'FRE': ['Freeport, ME', 43.8551159, -70.102229],
 'FRS': ['Springfield - Franconia, VA', 38.766411, -77.1682579],
 'FTC': ['Ticonderoga, NY', 43.8486707, -73.4234531],
 'FTL': ['Fort Lauderdale, FL', 26.1199417, -80.1698074],
 'FTN': ['Fulton, KY', 36.5042277, -88.8742259],
 'FTW': ['Fort Worth, TX', 32.7554883, -97.3307658],
 'FUL': ['Fullerton, CA', 33.8688442, -117.9228275],
 'GAC': ['Santa Clara - Great America, CA', 37.4064495, -121.9669427],
 'GAS': ['Gastonia, NC', 35.2692377, -81.1624033],
 'GBB': ['Galesburg, IL', 40.9446743, -90.3638544],
 'GCK': ['Garden City, KS', 37.9642526, -100.873401],
 'GDL': ['Glendale, CA', 34.1236087, -118.2591062],
 'GFD': ['Greenfield, MA', 42.587915, -72.5994104],
 'GFK': ['Grand Forks, ND', 47.9174444, -97.1108634],
 'GGW': ['Glasgow, MT', 48.1950417, -106.6358958],
 'GJT': ['Grand Junction, CO', 39.064566, -108.5705414],
 'GLE': ['Gainesville, TX', 33.6250844, -97.1407223],
 'GLM': ['Gilman, IL', 40.7667015, -87.992262],
 'GLN': ['Glenview, IL', 42.0749451, -87.8056388],
 'GLP': ['Gallup, NM', 35.5292234, -108.7399189],
 'GNB': ['Greensburg, PA', 40.3014581, -79.5389289],
 'GNS': ['Gainesville, GA', 34.2887701, -83.81962039999999],
 'GPK': ['East Glacier Park, MT', 48.442333, -113.219523],
 'GRA': ['Granby, CO', 40.0841503, -105.9356906],
 'GRI': ['Green River, UT', 38.99216089999999, -110.1653429],
 'GRO': ['Greensboro, NC', 36.0696217, -79.7869996],
 'GRR': ['Grand Rapids, MI', 42.9556863, -85.6723135],
 'GRV': ['Greenville, SC', 34.858508, -82.4135993],
 'GSC': ['Glenwood Springs, CO', 39.5480248, -107.3231577],
 'GTA': ['Goleta, CA', 34.4358294, -119.8276389],
 'GUA': ['Guadalupe-Santa Maria, CA', 34.9629051, -120.5734877],
 'GUT': ['Guthrie, OK', 35.8770379, -97.4300009],
 'GVB': ['Grover Beach, CA', 35.12126, -120.629266],
 'GWD': ['Greenwood, MS', 33.5171152, -90.1763225],
 'HAM': ['Hamlet, NC', 34.8836591, -79.699046],
 'HAR': ['Harrisburg, PA', 40.2622118, -76.8776627],
 'HAS': ['Hastings, NE', 40.5839584, -98.38764139999999],
 'HAV': ['Havre, MT', 48.5545804, -109.678352],
 'HAY': ['Hayward, CA', 37.6659621, -122.0993491],
 'HAZ': ['Hazlehurst, MS', 31.8613389, -90.3943192],
 'HBG': ['Hattiesburg, MS', 31.3270929, -89.2864075],
 'HEM': ['Hermann, MO', 38.7072959, -91.43251819999999],
 'HER': ['Helper, UT', 39.6839518, -110.8537682],
 'HFD': ['Hartford, CT', 41.76881909999999, -72.68157140000001],
 'HFY': ['Harpers Ferry, WV', 39.324584, -77.731023],
 'HGD': ['Huntingdon, PA', 40.48366499999999, -78.01178],
 'HHL': ['Haverhill, MA', 42.7762015, -71.0772796],
 'HIN': ['Hinton, WV', 37.6749618, -80.8922345],
 'HLD': ['Holdrege, NE', 40.43607060000001, -99.36991929999999],
 'HLK': ['Holyoke, MA', 42.2042586, -72.6162009],
 'HMD': ['Hammond, LA', 30.5071924, -90.4621933],
 'HMI': ['Hammond-Whiting, IN', 41.6910817, -87.506592],
 'HMW': ['Homewood, IL', 41.5620305, -87.6678472],
 'HNF': ['Hanford, CA', 36.3274502, -119.6456844],
 'HOL': ['Hollywood, FL', 26.0118963, -80.1678151],
 'HOM': ['Holland, MI', 42.7910765, -86.0964327],
 'HOP': ['Hope, AR', 33.66846719999999, -93.5971381],
 'HOS': ['Houston, TX', 29.7673689, -95.3675025],
 'HPT': ['High Point, NC', 35.9572214, -80.0061973],
 'HUD': ['Hudson, NY', 42.2540795, -73.7977148],
 'HUN': ['Huntington, WV', 38.4159452, -82.4397065],
 'HUT': ['Hutchinson, KS', 38.0556491, -97.9309526],
 'IDP': ['Independence, MO', 39.0868625, -94.4297684],
 'IND': ['Indianapolis, IN', 39.7624566, -86.1602383],
 'IRV': ['Irvine, CA', 33.6567687, -117.7335357],
 'JAN': ['Jackson, MS', 32.3009101, -90.1909628],
 'JAX': ['Jacksonville, FL', 30.36544009999999, -81.7240522],
 'JEF': ['Jefferson City, MO', 38.5788027, -92.17005809999999],
 'JOL': ['Joliet, IL', 41.525031, -88.0817251],
 'JSP': ['Jesup, GA', 31.6058753, -81.88192959999999],
 'JST': ['Johnstown, PA', 40.3298, -78.92183899999999],
 'JXN': ['Jackson, MI', 42.2162898, -84.3793452],
 'KAL': ['Kalamazoo, MI', 42.2954205, -85.5840685],
 'KAN': ['Kannapolis, NC', 35.4960531, -80.6247384],
 'KCY': ['Kansas City, MO', 39.0847966, -94.5853578],
 'KEE': ['Kewanee, IL', 41.2460006, -89.92757879999999],
 'KEL': ['Kelso-Longview, WA', 46.14219610000001, -122.9132561],
 'KFS': ['Klamath Falls, OR', 42.2255106, -121.7718459],
 'KIN': ['Kingston, RI', 41.4840417, -71.56067519999999],
 'KIS': ['Kissimmee, FL', 28.293714, -81.40423919999999],
 'KKI': ['Kankakee, IL', 41.1187624, -87.86602979999999],
 'KNG': ['Kingman, AZ', 35.189443, -114.0530065],
 'KTR': ['Kingstree, SC', 33.6636187, -79.829101],
 'KWD': ['Kirkwood, MO', 38.5809466, -90.4066363],
 'LAB': ['Latrobe, PA', 40.3180238, -79.3856765],
 'LAF': ['Lafayette, IN', 40.4197241, -86.8957303],
 'LAG': ['La Grange Road, IL', 41.8157562, -87.8710321],
 'LAJ': ['La Junta, CO', 37.9883529, -103.5431431],
 'LAK': ['Lakeland - Departure 92 North Arrival 91 South, FL', 27.6648274, -81.5157535],
 'LAP': ['La Plata, MO', 40.0291494, -92.4938727],
 'LAU': ['Laurel, MS', 31.6923213, -89.12799609999999],
 'LAX': ['Los Angeles - Union Station, CA', 34.0559225, -118.2349932],
 'LCH': ['Lake Charles, LA', 30.23816, -93.21811269999999],
 'LCN': ['Lincoln, IL', 40.1481045, -89.3632796],
 'LDB': ['Lordsburg, NM', 32.35007, -108.707025],
 'LEE': ['Lees Summit, MO', 38.9126791, -94.3783309],
 'LEW': ['Lewistown, PA', 40.5882076, -77.58034590000001],
 'LEX': ['Lexington, NC', 35.8240265, -80.2533838],
 'LFT': ['Lafayette, LA', 30.2267084, -92.01425359999999],
 'LIB': ['Libby, MT', 48.3949343, -115.5489125],
 'LKL': ['Lakeland - Departure 91 North Arrival 92 South, FL', 28.0457559, -81.95188619999999],
 'LMR': ['Lamar, CO', 38.0897462, -102.6184191],
 'LMY': ['Lamy, NM', 35.480973, -105.879968],
 'LNC': ['Lancaster, PA', 40.0378755, -76.3055144],
 'LNK': ['Lincoln, NE', 40.815886, -96.71393979999999],
 'LNL': ['Laguna Niguel - Mission Viejo, CA', 33.5531321, -117.6740694],
 'LNS': ['East Lansing, MI', 42.7189349, -84.4958936],
 'LOD': ['Lodi, CA', 38.1331996, -121.2718086],
 'LOR': ['Lorton - Auto Train Only, VA', 38.7084193, -77.2206046],
 'LPE': ['Lapeer, MI', 43.049443, -83.30615809999999],
 'LPS': ['Lompoc-Surf, CA', 34.6829861, -120.6049955],
 'LRC': ['Lawrence, KS', 38.971265, -95.230492],
 'LRK': ['Little Rock, AR', 34.7444618, -92.2880157],
 'LSE': ['La Crosse, WI', 43.8332547, -91.2472282],
 'LSV': ['Las Vegas, NM', 35.5942137, -105.2227978],
 'LVW': ['Longview, TX', 32.4941135, -94.72780910000002],
 'LWA': ['Leavenworth, WA', 47.5962326, -120.6614765],
 'LYH': ['Lynchburg, VA', 37.4064059, -79.1571274],
 'MAC': ['Macomb, IL', 40.4613757, -90.67098399999999],
 'MAL': ['Malta, MT', 48.3604236, -107.8719094],
 'MAT': ['Mattoon, IL', 39.4830146, -88.3758622],
 'MAY': ['Maysville, KY', 38.6521592, -83.771109],
 'MBY': ['Middlebury, VT', 44.017669, -73.16985729999999],
 'MCB': ['McComb, MS', 31.2445654, -90.451246],
 'MCD': ['Merced, CA', 37.3073155, -120.4767677],
 'MCG': ['McGregor, TX', 31.4433297, -97.4050061],
 'MCI': ['Michigan City, IN', 41.72114029999999, -86.9054694],
 'MCK': ['McCook, NE', 40.1975153, -100.6256753],
 'MDN': ['Meriden, CT', 41.5381535, -72.80704349999999],
 'MDR': ['Madera, CA', 37.022576, -120.0751248],
 'MDT': ['Mendota, IL', 41.5497518, -89.11794259999999],
 'MEI': ['Meridian, MS', 32.3639846, -88.69648459999999],
 'MEM': ['Memphis, TN', 35.1324021, -90.05942019999999],
 'MET': ['Metropark-Iselin, NJ', 40.568101, -74.3296338],
 'MHL': ['Marshall, TX', 32.5519955, -94.36755360000001],
 'MIA': ['Miami, FL', 25.8494665, -80.2579847],
 'MID': ['Middletown, PA', 40.1926543, -76.73112680000001],
 'MIN': ['Mineola, TX', 32.6621141, -95.4890611],
 'MJY': ['Mount Joy, PA', 40.1089701, -76.50355619999999],
 'MKA': ['Milwaukee - Airport, WI', 43.0389025, -87.9064736],
 'MKE': ['Milwaukee, WI', 43.0343825, -87.9174031],
 'MKS': ['Marks, MS', 34.258227, -90.272342],
 'MNG': ['Montgomery, WV', 38.1807527, -81.3239602],
 'MOD': ['Modesto, CA', 37.6690246, -120.9127369],
 'MOT': ['Minot, ND', 48.2360992, -101.2986583],
 'MPK': ['Moorpark, CA', 34.2848664, -118.8780516],
 'MPR': ['Montpelier-Barre, VT', 44.2556664, -72.60637249999999],
 'MRB': ['Martinsburg, WV', 39.4586105, -77.96089479999999],
 'MRC': ['Maricopa, AZ', 33.056128, -112.0475271],
 'MSP': ['St. Paul / Minneapolis, MN', 44.94703, -93.0854459],
 'MSS': ['Manassas, VA', 38.7501193, -77.472811],
 'MTP': ['Mount Pleasant, IA', 40.9712234, -91.5506355],
 'MTR': ['Montreal, QC', 45.5018869, -73.56739189999999],
 'MTZ': ['Martinez, CA', 38.0193657, -122.1341321],
 'MVN': ['Malvern, AR', 34.3656736, -92.81373239999999],
 'MVW': ['Mount Vernon, WA', 48.4185028, -122.3349677],
 'MYS': ['Mystic, CT', 41.3509332, -71.9632165],
 'NBK': ['New Brunswick, NJ', 40.496654, -74.446092],
 'NBM': ['New Buffalo, MI (old)', 41.7939302, -86.74391539999999],
 'NBN': ['Newbern, TN', 36.1128479, -89.2617368],
 'NBU': ['New Buffalo, MI', 41.7939302, -86.74391539999999],
 'NCR': ['New Carrollton, MD', 38.948407, -76.872084],
 'NDL': ['Needles, CA', 34.8406307, -114.6059022],
 'NEW': ['Newton, KS', 38.047121, -97.3445546],
 'NFK': ['Norfolk, VA', 36.8434849, -76.2760966],
 'NFL': ['Niagara Falls, NY', 43.10988770000001, -79.0549276],
 'NFS': ['Niagara Falls, ON', 43.10988770000001, -79.0549276],
 'NHT': ['Northampton, MA', 42.3188531, -72.6264399],
 'NHV': ['New Haven, CT', 41.308274, -72.9278835],
 'NIB': ['New Iberia, LA', 30.008415, -91.823809],
 'NLC': ['New London, CT', 41.3556539, -72.0995209],
 'NLS': ['Niles, MI', 41.8373369, -86.25229039999999],
 'NOL': ['New Orleans, LA', 29.95106579999999, -90.0715323],
 'NOR': ['Norman, OK', 35.2198854, -97.4428739],
 'NPN': ['Newport News, VA', 37.0228094, -76.4518617],
 'NPV': ['Naperville, IL', 41.7752632, -88.1446452],
 'NRG': ['Northridge, CA', 34.2381251, -118.530123],
 'NRK': ['Newark, DE', 39.6696924, -75.7535438],
 'NRO': ['New Rochelle, NY', 40.9114981, -73.7841466],
 'NSF': ['North Carolina State Fair, NC', 35.7940226, -78.7062868],
 'NWK': ['Newark, NJ', 40.7043916, -74.19070789999999],
 'NYF': ['New York State Fair - Syracuse, NY', 43.074131, -76.2217178],
 'NYG': ['New York - Grand Central Terminal, NY', 40.7527262, -73.9772294],
 'NYP': ['New York - Penn Station, NY', 40.750568, -73.993519],
 'OAC': ['Oakland - Coliseum Airport, CA', 37.7515946, -122.2005458],
 'OGE': ['Orange, CA', 33.7887799, -117.8574039],
 'OKC': ['Oklahoma City, OK', 35.4653748, -97.51262960000001],
 'OKE': ['Okeechobee, FL', 27.2439349, -80.82978279999999],
 'OKJ': ['Oakland - Jack London Sq, CA', 37.79372, -122.27152],
 'OLT': ['San Diego - Old Town, CA', 32.75523039999999, -117.1995263],
 'OLW': ['Olympia-Lacey, WA', 46.9913448, -122.7939059],
 'OMA': ['Omaha, NE', 41.2497765, -95.92717660000001],
 'ONA': ['Ontario, CA', 34.06163, -117.649595],
 'ORB': ['Old Orchard Beach, ME', 43.5177659, -70.3772911],
 'ORC': ['Oregon City, OR', 45.3659511, -122.5960694],
 'ORL': ['Orlando, FL', 28.5265366, -81.3818149],
 'OSB': ['Old Saybrook, CT', 41.3004331, -72.37752569999999],
 'OSC': ['Osceola, IA', 41.037178, -93.76499629999999],
 'OSD': ['Oceanside, CA', 33.1915794, -117.356686],
 'OSF': ['Salem - Oregon State Fairgrounds, OR', 44.9428975, -123.0350963],
 'OTM': ['Ottumwa, IA', 41.0186748, -92.41493080000001],
 'OXN': ['Oxnard, CA', 34.1994251, -119.1761666],
 'PAK': ['Palatka, FL', 29.6485801, -81.6375819],
 'PAO': ['Paoli, PA', 40.0428278, -75.4836423],
 'PAR': ['Parkesburg, PA', 39.9590234, -75.9224655],
 'PBF': ['Poplar Bluff, MO', 36.7539046, -90.3933649],
 'PCA': ['Ponca City, OK', 36.7065015, -97.0844845],
 'PCT': ['Princeton, IL', 41.38523910000001, -89.4667765],
 'PDX': ['Portland - Union Station, OR', 45.5290829, -122.6767744],
 'PGH': ['Pittsburgh, PA', 40.4446786, -79.9915525],
 'PHL': ['Philadelphia - 30th Street Station, PA', 39.9567993, -75.1822703],
 'PHN': ['North Philadelphia, PA', 39.9971867, -75.15525160000001],
 'PIC': ['Picayune, MS', 30.5254692, -89.67950839999999],
 'PIT': ['Pittsfield, MA', 42.4514194, -73.2540964],
 'PJC': ['Princeton Junction, NJ', 40.3173301, -74.61987909999999],
 'PLB': ['Plattsburgh, NY', 44.6967124, -73.44647970000001],
 'PLO': ['Plano, IL', 41.6622583, -88.5383056],
 'PNT': ['Pontiac, MI', 42.6326185, -83.292175],
 'POG': ['Portage, WI', 43.5470213, -89.4672018],
 'POH': ['Port Henry, NY', 44.0423696, -73.4588407],
 'POI': ['Carlsbad - Poinsettia, CA', 33.1609289, -117.3508223],
 'PON': ['Pontiac, IL', 40.8787723, -88.6370576],
 'POR': ['Portland, ME', 43.6539728, -70.2913009],
 'POS': ['Pomona, CA', 34.055103, -117.7499909],
 'POU': ['Poughkeepsie, NY', 41.7003713, -73.9209701],
 'PRB': ['Paso Robles, CA', 35.6226713, -120.6877691],
 'PRC': ['Prince, WV', 37.856506, -81.0606181],
 'PRK': ['Port Kent, NY', 44.5239688, -73.4034396],
 'PRO': ['Provo, UT', 40.2254153, -111.6604034],
 'PRY': ['Perry, OK', 36.2894855, -97.28809830000002],
 'PSC': ['Pasco, WA', 46.2369283, -119.087476],
 'PSN': ['Palm Springs - North, CA', 33.8977419, -116.5481285],
 'PTB': ['Petersburg, VA', 37.2419252, -77.4286938],
 'PTH': ['Port Huron, MI', 42.9708634, -82.42491419999999],
 'PUR': ['Purcell, OK', 35.012033, -97.357416],
 'PVD': ['Providence, RI', 41.8292098, -71.4135513],
 'PVL': ['Pauls Valley, OK', 34.7400809, -97.22224510000001],
 'QAN': ['Quantico, VA', 38.5229229, -77.2900787],
 'QCY': ['Quincy, IL', 39.9356016, -91.4098726],
 'RAT': ['Raton, NM', 36.9009612, -104.4379354],
 'RDD': ['Redding, CA', 40.5837272, -122.3933311],
 'RDW': ['Red Wing, MN', 44.56633619999999, -92.53706919999999],
 'REN': ['Rensselaer, IN', 40.943281, -87.155121],
 'RGH': ['Raleigh, NC', 35.7795897, -78.6381787],
 'RHI': ['Rhinecliff, NY', 41.9200125, -73.9524332],
 'RIC': ['Richmond, CA', 37.93677, -122.3539],
 'RIV': ['Riverside, CA', 33.9806005, -117.3754942],
 'RKV': ['Rockville, MD', 39.0839973, -77.1527578],
 'RLN': ['Rocklin, CA', 38.791006, -121.237299],
 'RMT': ['Rocky Mount, NC', 35.9379596, -77.79767609999999],
 'RNK': ['Roanoke, VA', 37.2731343, -79.94148799999999],
 'RNO': ['Reno, NV', 39.5287235, -119.8115627],
 'ROC': ['Rochester, NY', 43.1635068, -77.6081859],
 'ROM': ['Rome, NY', 43.1995334, -75.4498403],
 'ROY': ['Royal Oak, MI', 42.489273, -83.1476134],
 'RPH': ['Randolph, VT', 43.9224688, -72.6654855],
 'RSP': ['Rouses Point, NY', 44.99506, -73.3712202],
 'RSV': ['Roseville, CA', 38.7521235, -121.2880059],
 'RTE': ['Westwood - Route 128, MA', 42.2103494, -71.1474904],
 'RTL': ['Rantoul, IL', 40.3109104, -88.15920419999999],
 'RUD': ['Rutland, VT', 43.6057603, -72.981752],
 'RUG': ['Rugby, ND', 48.3696911, -99.9976971],
 'RVM': ['Richmond - Main Street Station, VA', 37.5351005, -77.4287069],
 'RVR': ['Richmond - Staples Mill Rd, VA', 37.61734089999999, -77.49761869999999],
 'SAB': ['St. Albans, VT', 44.8107132, -73.0835582],
 'SAC': ['Sacramento, CA', 38.58439480000001, -121.5006613],
 'SAL': ['Salisbury, NC', 35.6673631, -80.4661983],
 'SAN': ['San Diego, CA', 32.7168533, -117.1695457],
 'SAO': ['Saco, ME', 43.496057, -70.4492051],
 'SAR': ['Saratoga Springs, NY', 43.0821738, -73.8099596],
 'SAS': ['San Antonio, TX', 29.4193893, -98.47804230000001],
 'SAV': ['Savannah, GA', 32.0836192, -81.1484144],
 'SBA': ['Santa Barbara, CA', 34.413654, -119.692914],
 'SBG': ['Sebring, FL', 27.4970594, -81.4344392],
 'SBY': ['Shelby, MT', 48.5068434, -111.8571515],
 'SCC': ['Santa Clara, CA', 37.3541079, -121.9552356],
 'SCD': ['St. Cloud, MN', 45.5678311, -94.14895090000002],
 'SCH': ['Schriever, LA', 29.7465736, -90.8155407],
 'SDL': ['Slidell, LA', 30.2784417, -89.7825541],
 'SDY': ['Schenectady, NY', 42.8145664, -73.94273079999999],
 'SEA': ['Seattle - King Street Station, WA', 47.5983889, -122.3299086],
 'SED': ['Sedalia, MO', 38.7115378, -93.22827190000001],
 'SFA': ['Sanford - Auto Train Only, FL', 28.8087842, -81.2914469],
 'SIM': ['Simi Valley, CA', 34.2705084, -118.6958361],
 'SJC': ['San Jose, CA', 37.3298351, -121.9026401],
 'SJM': ['St. Joseph, MI', 42.1091515, -86.4844734],
 'SKN': ['Stockton, CA', 37.9453315, -121.2856017],
 'SKT': ['Stockton - ACE Station, CA', 37.9577016, -121.2907796],
 'SKY': ['Sandusky, OH', 41.4406713, -82.7186744],
 'SLC': ['Salt Lake City, UT', 40.7617285, -111.908466],
 'SLM': ['Salem, OR', 44.932339, -123.0282148],
 'SLO': ['San Luis Obispo, CA', 35.2765252, -120.6545691],
 'SLQ': ['St. Lambert, QC', 45.4990444, -73.5071964],
 'SMC': ['San Marcos, TX', 29.8765145, -97.9411877],
 'SMT': ['Summit, IL', 41.7949693, -87.80965619999999],
 'SNA': ['Santa Ana, CA', 33.75157660000001, -117.8564385],
 'SNB': ['San Bernardino, CA', 34.1042433, -117.3099262],
 'SNC': ['San Juan Capistrano, CA', 33.5019027, -117.6627655],
 'SND': ['Sanderson, TX', 30.1402323, -102.3994996],
 'SNP': ['San Clemente, CA', 33.419645, -117.619733],
 'SNS': ['Salinas, CA', 36.6777372, -121.6555013],
 'SOB': ['South Bend, IN', 41.6782607, -86.2874289],
 'SOL': ['Solana Beach, CA', 32.9929837, -117.2712791],
 'SOP': ['Southern Pines, NC', 35.175069, -79.390303],
 'SPB': ['Spartanburg, SC', 34.9535248, -81.93751569999999],
 'SPG': ['Springfield, MA', 42.1065267, -72.593643],
 'SPI': ['Springfield, IL', 39.802343, -89.6515763],
 'SPK': ['Spokane, WA', 47.6563567, -117.4153822],
 'SPL': ['Staples, MN', 46.354501, -94.7954862],
 'SPM': ['South Portsmouth - South Shore, KY', 38.721281, -82.963751],
 'SPR': ['Sparks, NV', 39.534638, -119.761757],
 'SPT': ['Sandpoint, ID', 48.2764006, -116.5456166],
 'SRB': ['San Diego - Sorrento Valley, CA', 32.9000226, -117.1894828],
 'SSM': ['Selma-Smithfield, NC', 35.5326438, -78.2801072],
 'STA': ['Staunton, VA', 38.1477991, -79.0724505],
 'STL': ['St. Louis, MO', 38.624216, -90.2036115],
 'STM': ['Stamford, CT', 41.0469065, -73.5420051],
 'STN': ['Stanley, ND', 48.3199022, -102.3894996],
 'STS': ['New Haven - State Street Station, CT', 41.3052279, -72.9220283],
 'STW': ['Stanwood, WA', 48.24261620000001, -122.3500778],
 'SUI': ['Suisun-Fairfield, CA', 38.24345, -122.04119],
 'SVT': ['Sturtevant, WI', 42.718062, -87.906161],
 'SYR': ['Syracuse, NY', 43.0481221, -76.14742439999999],
 'TAC': ['Tacoma, WA', 47.2395287, -122.4280727],
 'TAY': ['Taylor, TX', 30.5701906, -97.4097331],
 'TCA': ['Toccoa, GA', 34.57844559999999, -83.3318225],
 'TCL': ['Tuscaloosa, AL', 33.193412, -87.5601474],
 'THN': ['Thurmond, WV', 37.957115, -81.078813],
 'TMV': ['Thomasville, NC', 35.8826369, -80.0819879],
 'TOH': ['Tomah, WI', 43.9785759, -90.50402140000001],
 'TOL': ['Toledo, OH', 41.6381844, -83.5408551],
 'TOP': ['Topeka, KS', 39.0513976, -95.6648171],
 'TPA': ['Tampa, FL', 27.9528043, -82.4508239],
 'TPL': ['Temple, TX', 31.0957286, -97.3452969],
 'TRE': ['Trenton, NJ', 40.2205824, -74.759717],
 'TRI': ['Trinidad, CO', 37.1722943, -104.5084615],
 'TRK': ['Turlock-Denair, CA', 37.52726639999999, -120.7980756],
 'TRM': ['Troy, MI', 42.5425914, -83.19089029999999],
 'TRU': ['Truckee, CA', 39.3274982, -120.1856766],
 'TUK': ['Tukwila, WA', 47.460666, -122.2405258],
 'TUS': ['Tucson, AZ', 32.2231036, -110.9667129],
 'TWO': ['Toronto, ON', 43.653226, -79.3831843],
 'TXA': ['Texarkana, AR', 33.420294, -94.04217179999999],
 'TYR': ['Tyrone, PA', 40.6676672, -78.24047879999999],
 'UCA': ['Utica, NY', 43.1041839, -75.2232219],
 'VAC': ['Vancouver - Pacific Central Station, BC', 49.2736975, -123.0978918],
 'VAN': ['Vancouver, WA', 45.6287601, -122.6866133],
 'VEC': ['Ventura, CA', 34.2769475, -119.2998742],
 'VNC': ['Van Nuys, CA', 34.2113827, -118.4483004],
 'VRN': ['Ferrisburgh-Vergennes, VT', 44.1808944, -73.2488482],
 'VRV': ['Victorville, CA', 34.5373956, -117.2937236],
 'WAB': ['Waterbury-Stowe, VT', 44.3378203, -72.7562636],
 'WAC': ['Wasco, CA', 35.5942252, -119.3321207],
 'WAH': ['Washington, MO', 38.5615745, -91.01245970000001],
 'WAR': ['Warrensburg, MO', 38.7628115, -93.74090249999999],
 'WAS': ['Washington - Union Station, DC', 38.8977859, -77.0057621],
 'WBG': ['Williamsburg, VA', 37.2765426, -76.7086559],
 'WDB': ['Woodbridge, VA', 38.6590458, -77.2481554],
 'WDL': ['Wisconsin Dells, WI', 43.6266317, -89.7775338],
 'WEM': ['Wells, ME', 43.3207824, -70.6121422],
 'WEN': ['Wenatchee, WA', 47.421646, -120.306549],
 'WFD': ['Wallingford, CT', 41.4616113, -72.8227914],
 'WFH': ['Whitefish, MT', 48.4135219, -114.3358981],
 'WGL': ['West Glacier, MT', 48.4949755, -113.9810756],
 'WHL': ['Whitehall, NY', 43.5547552, -73.40321209999999],
 'WIC': ['Wichita, KS', 37.68717609999999, -97.33005299999999],
 'WIH': ['Wishram, WA', 45.65765649999999, -120.9663977],
 'WIL': ['Wilmington, DE', 39.7370981, -75.5508771],
 'WIN': ['Winona, MN', 44.0553908, -91.6663523],
 'WIP': ['Fraser, CO', 39.9476113, -105.8176213],
 'WLN': ['Wilson, NC', 35.7231863, -77.90787929999999],
 'WLO': ['Winslow, AZ', 35.0241873, -110.6973571],
 'WLY': ['Westerly, RI', 41.38119409999999, -71.82986120000001],
 'WMJ': ['Williams Junction, AZ', 35.2425875, -112.1318239],
 'WND': ['Windsor, CT', 41.8521506, -72.642292],
 'WNL': ['Windsor Locks, CT', 41.9137841, -72.6262942],
 'WNM': ['Windsor, VT', 43.4799996, -72.3865679],
 'WNN': ['Winnemucca, NV', 40.9688487, -117.7323409],
 'WNR': ['Walnut Ridge, AR', 36.067532, -90.9567458],
 'WOB': ['Woburn, MA', 42.4792618, -71.1522765],
 'WOR': ['Worcester, MA', 42.2612345, -71.7949679],
 'WPB': ['West Palm Beach, FL', 26.7132739, -80.0622896],
 'WPK': ['Winter Park, FL', 28.5999998, -81.33923519999999],
 'WPR': ['Winter Park - Ski Resort, CO', 39.8841389, -105.7627026],
 'WPT': ['Wolf Point, MT', 48.09152599999999, -105.6430527],
 'WRJ': ['White River Junction, VT', 43.6478037, -72.3175186],
 'WSP': ['Westport, NY', 44.18338259999999, -73.4637448],
 'WSS': ['White Sulphur Springs, WV', 37.7866167, -80.3038291],
 'WTH': ['Winter Haven, FL', 28.0020239, -81.7348848],
 'WTI': ['Waterloo, IN', 41.4317561, -85.0242901],
 'WTN': ['Williston, ND', 48.142851, -103.6211029],
 'XXX': [''],
 'YAZ': ['Yazoo City, MS', 32.8484289, -90.4151631],
 'YEM': ['Yemassee, SC', 32.6880438, -80.84704119999999],
 'YNY': ['Yonkers, NY', 40.9312099, -73.89874689999999],
 'YUM': ['Yuma, AZ', 32.7229832, -114.615562]
}   

# Great Circle Distance

In [23]:
def greatCircleDistanceMiles(lat1, long1, lat2, long2):
    lat1 = lat1 * math.pi / 180
    long1 = long1 * math.pi / 180
    lat2 = lat2 * math.pi / 180
    long2 = long2 * math.pi / 180
    distance = 3958.8 * math.acos(math.sin(lat1) * math.sin(lat2) + math.cos(lat1) * math.cos(lat2) * math.cos(long1 - long2))
    return distance

# Get Distance for Each Train

In [24]:
def getRouteDistances():    
    for train in trainDict:
        lat1 = stationCoordinates[trainDict[train][0]][1]
        long1 = stationCoordinates[trainDict[train][0]][2]
        lat2 = stationCoordinates[trainDict[train][1]][1]
        long2 = stationCoordinates[trainDict[train][1]][2]
        distance = greatCircleDistanceMiles(lat1, long1, lat2, long2)
        if len(trainDict[train]) == 2:
            trainDict[train].append(distance)
        else:
            trainDict[train][2] = distance
    

In [25]:
getRouteDistances()

# Get Stats for Each Train Route

In [26]:
# Get the stats we need from the files
def getStats(folder):
  
    medianDict = { }
    meanDict = { }
    medianDictNormalized = { }
    meanDictNormalized = { }
    varDict = { }
    delaysByTrain = { }
    
    # Look through all trains
    for filename in os.listdir(folder):
    
        train = filename.find("Train")
        i = filename[train+5:len(filename)-4]
        i = int(i)
        
        filepath = folder + "/" + filename
        file = open(filepath, "r")
        text = file.read()

        delaysByTrain[i] = []
        
        toFind = "</span></td><td style=\"text-align: center;\">"
        length = len(toFind)
        searching = text.find(toFind)
        while (searching != -1) :
            end = text.find("<", searching + length)
            number = int(text[searching+length:end])
            if number > -130:
                delaysByTrain[i].append(number)
            searching = text.find(toFind, end)
    
    for x in routeTrains:
        listDelays = []
        for y in routeTrains[x]:
            listDelays = listDelays + delaysByTrain[y]
        if len(listDelays) >= 0:
            i = x[0] + " to " + x[1]
            delays = np.asarray(listDelays)
            median = np.median(delays)
            medianDict[i] = []
            medianDict[i].append(median)
            medianDict[i].append(len(listDelays))
            medianDictNormalized[i] = []
            medianDictNormalized[i].append(median / trainDict[int(routeTrains[x][0])][2])
            medianDictNormalized[i].append(len(listDelays))
            mean = np.mean(delays)
            meanDict[i] = []
            meanDict[i].append(mean)
            meanDict[i].append(len(listDelays))
            meanDictNormalized[i] = []
            meanDictNormalized[i].append(mean / trainDict[int(routeTrains[x][0])][2])
            meanDictNormalized[i].append(len(listDelays))
            var = np.var(delays)
            varDict[i] = []
            varDict[i].append(var)
            varDict[i].append(len(listDelays))
        
    return medianDict, meanDict, medianDictNormalized, meanDictNormalized, varDict

In [27]:
folder2 = "2021_2022_DestinationDelays"
medianDict, meanDict, medianDictNormalized, meanDictNormalized, varDict = getStats(folder2)
# print(medianDict)

# List Train Routes

In [28]:
# Top 5 trains sorted by Median Arrival Delay Time
medSort = dict(sorted(medianDict.items(), key=lambda item: item[1]))
print("Median (Minutes Delayed)")
print()
print("{}\t\t{}\t{}".format("Route","Minutes","Sample Size"))
count = 0
for i in medSort:
    if medSort[i][1] >= 52:
        print("{}\t{}\t{}".format(i,medSort[i][0].round(3),medSort[i][1]))
        count = count + 1
        # if count == 5:
            # break
# Could view full list, output is very long

Median (Minutes Delayed)

Route		Minutes	Sample Size
CHI to NOL	-46.0	539
CHI to SEA	-19.5	572
NOL to CHI	-19.0	592
SEA to LAX	-15.0	576
TWO to NYP	-11.0	716
CHI to PTH	-10.0	709
OAC to BFD	-9.0	723
RNK to BOS	-9.0	1028
QCY to CHI	-8.0	1180
OKJ to SAC	-8.0	2360
CHI to CDL	-6.0	922
CHI to STL	-6.0	2185
SJC to ARN	-5.0	630
CHI to QCY	-5.0	1183
NFL to NYP	-5.0	1442
PHL to HAR	-5.0	1587
SJC to SAC	-5.0	3939
CLT to NYP	-4.5	706
RUD to NYP	-4.0	718
GTA to SAN	-4.0	1258
SAC to OKJ	-4.0	2363
LAX to SAN	-3.0	3425
SAC to SJC	-3.0	3720
PHL to NYP	-2.0	111
CHI to GRR	-2.0	714
CDL to CHI	-2.0	919
STL to CHI	-2.0	2171
ALB to NYP	-2.0	3434
GFD to NHV	-1.5	1000
SPG to GFD	-1.0	216
SPG to WAS	-1.0	634
SPG to NHV	-1.0	2619
OKJ to BFD	-1.0	2694
BFD to OKJ	-1.0	3357
SAN to LAX	-1.0	3545
MET to BOS	0.0	175
SAC to BFD	0.0	431
SPK to PDX	0.0	578
SAB to WAS	0.0	645
GRR to CHI	0.0	715
NHV to GFD	0.0	841
MKE to CHI	0.0	4293
CHI to MKE	0.0	4303
HAR to NYP	0.0	4314
NPN to NYP	1.0	168
OSD to SAN	1.0	348
SAV to NYP

In [29]:
# Top 5 trains sorted by Median Arrival Delay Time Normalized By Distance
medSortNormalized = dict(sorted(medianDictNormalized.items(), key=lambda item: item[1]))
print("Median (Minutes Delayed per Mile)")
print()
print("{}\t\t{}\t{}".format("Route","Minutes","Sample Size"))
count = 0
for i in medSortNormalized:
    if medSortNormalized[i][1] >= 52:
        count = count + 1
        print("{}\t{}\t{}\t{}".format(count,i,medSortNormalized[i][0].round(3),medSortNormalized[i][1]))
        # if count == 5:
            # break
# Could view full list, output is very long

Median (Minutes Delayed per Mile)

Route		Minutes	Sample Size
1	OKJ to SAC	-0.116	2360
2	SAC to OKJ	-0.058	2363
3	CHI to NOL	-0.055	539
4	CHI to PTH	-0.036	709
5	SAC to SJC	-0.034	3720
6	LAX to SAN	-0.027	3425
7	CHI to STL	-0.023	2185
8	NOL to CHI	-0.023	592
9	CHI to QCY	-0.021	1183
10	CHI to CDL	-0.02	922
11	CHI to GRR	-0.016	714
12	SEA to LAX	-0.016	576
13	CHI to SEA	-0.011	572
14	SAC to BFD	0.0	431
15	SPK to PDX	0.0	578
16	CHI to MKE	0.0	4303
17	NOL to NYP	0.002	562
18	CHI to NYP	0.006	886
19	CHI to SAS	0.017	587
20	CHI to PNT	0.022	1638
21	CHI to BOS	0.022	526
22	SEA to CHI	0.024	579
23	LAX to CHI	0.031	589
24	CHI to LAX	0.034	589
25	LAX to SEA	0.036	585
26	LAX to NOL	0.045	292
27	CHI to EMY	0.047	592
28	EMY to CHI	0.051	598
29	NOL to LAX	0.052	303
30	SEA to EUG	0.053	1267
31	CHI to WAS	0.067	616
32	SEA to PDX	0.076	579
33	CHI to KCY	0.113	214
34	SEA to VAC	0.194	96
35	NYP to RUD	-0.064	59
36	SJC to SAC	-0.056	3939
37	PHL to HAR	-0.054	1587
38	BOS to ALB	-0.051	536
39	SJC to ARN	-0

In [30]:
# Top 5 Trains sorted by Mean Arrival Delay Time
meanSort = dict(sorted(meanDict.items(), key=lambda item: item[1]))
print("Mean (Minutes Delayed)")
print()
print("{}\t{}\t\t{}".format("Route","Minutes","Sample Size"))
count = 0
for i in meanSort:
    if meanSort[i][1] >= 52:
        print("{}\t{}\t{}".format(i,meanSort[i][0].round(3),meanSort[i][1]))
        count = count + 1
        # if count == 5:
            # break
# Could view full list, output is very long

Mean (Minutes Delayed)

Route	Minutes		Sample Size
CHI to NOL	-16.642	539
PHL to HAR	-3.224	1587
OKJ to SAC	-3.144	2360
TWO to NYP	-1.983	716
QCY to CHI	-1.257	1180
NFL to NYP	-0.353	1442
GFD to NHV	-0.002	1000
OAC to BFD	0.091	723
PHL to NYP	0.532	111
NOL to CHI	0.688	592
SAC to OKJ	0.81	2363
SPG to NHV	0.98	2619
ALB to NYP	1.113	3434
SPG to GFD	1.282	216
SJC to SAC	1.31	3939
RNK to BOS	1.697	1028
MKE to CHI	1.832	4293
RUD to NYP	2.001	718
SAC to SJC	2.227	3720
HAR to NYP	2.351	4314
CHI to MKE	2.373	4303
CHI to QCY	2.686	1183
SJC to ARN	2.954	630
LAX to SAN	3.163	3425
CHI to CDL	3.773	922
GTA to SAN	3.841	1258
CDL to CHI	3.854	919
HAR to PHL	3.878	1154
SAN to OSD	4.172	435
SAN to LAX	4.412	3545
CHI to STL	4.47	2185
CHI to GRR	5.829	714
NHV to SPG	5.879	2574
STL to CHI	6.031	2171
SPG to WAS	6.338	634
MET to BOS	6.531	175
GRR to CHI	7.008	715
CLT to NYP	7.263	706
BFD to OKJ	7.301	3357
ARN to SJC	7.337	722
OSD to SAN	7.417	348
SAC to BFD	7.418	431
SLO to SAN	7.554	679
OKJ to BFD	7.747	26

In [31]:
# Top 5 Trains sorted by Mean Arrival Delay Time
meanSortNormalized = dict(sorted(meanDictNormalized.items(), key=lambda item: item[1]))
print("Mean (Minutes Delayed Per Mile)")
print()
print("{}\t\t{}\t{}".format("Route","Minutes","Sample Size"))
count = 0
for i in meanSortNormalized:
    if meanSortNormalized[i][1] >= 52:
        print("{}\t{}\t{}".format(i,meanSortNormalized[i][0].round(3),meanSortNormalized[i][1]))
        count = count + 1
        # if count == 5:
            # break
# Could view full list, output is very long

Mean (Minutes Delayed Per Mile)

Route		Minutes	Sample Size
OKJ to SAC	-0.046	2360
CHI to NOL	-0.02	539
NOL to CHI	0.001	592
CHI to QCY	0.011	1183
SAC to OKJ	0.012	2363
CHI to SEA	0.012	572
CHI to CDL	0.013	922
CHI to STL	0.017	2185
SEA to LAX	0.021	576
SAC to SJC	0.025	3720
NOL to NYP	0.028	562
SAC to BFD	0.028	431
LAX to SAN	0.028	3425
CHI to MKE	0.029	4303
CHI to PTH	0.038	709
CHI to SAS	0.045	587
CHI to NYP	0.045	886
CHI to BOS	0.046	526
CHI to GRR	0.047	714
SEA to CHI	0.056	579
LAX to CHI	0.058	589
CHI to LAX	0.059	589
LAX to SEA	0.07	585
LAX to NOL	0.071	292
CHI to EMY	0.072	592
EMY to CHI	0.076	598
NOL to LAX	0.08	303
CHI to PNT	0.083	1638
CHI to WAS	0.099	616
SEA to EUG	0.101	1267
SPK to PDX	0.138	578
SEA to PDX	0.142	579
CHI to KCY	0.151	214
SEA to VAC	0.25	96
NYP to RUD	-0.055	59
PHL to HAR	-0.035	1587
TWO to NYP	-0.006	716
QCY to CHI	-0.005	1180
NYP to HAR	-0.002	4650
NFL to NYP	-0.001	1442
GFD to NHV	-0.0	1000
OAC to BFD	0.0	723
RNK to BOS	0.003	1028
PHL to NYP	0.006	111
AL

In [32]:
# Top 5 Trains sorted by Variance of Arrival Delay Time
varSort = dict(sorted(varDict.items(), key=lambda item: item[1]))
print("Standard Deviation")
print()
print("{}\t\t{}\t{}".format("Route","Minutes","Sample Size"))
count = 0
for i in varSort:
    if varSort[i][1] >= 52:
        print("{}\t{}\t{}".format(i,np.sqrt(varSort[i][0]).round(3),varSort[i][1]))
        count = count + 1
        # if count == 5:
            # break
# Could view full list, output is very long

Standard Deviation

Route		Minutes	Sample Size
CHI to MKE	13.602	4303
SAC to OKJ	15.035	2363
LAX to SAN	19.837	3425
SAC to SJC	21.286	3720
CHI to GRR	26.099	714
SAC to BFD	26.63	431
CHI to QCY	27.307	1183
SEA to VAC	35.031	96
CHI to STL	35.622	2185
SEA to PDX	38.883	579
CHI to CDL	42.466	922
SEA to EUG	43.616	1267
CHI to PTH	50.672	709
CHI to PNT	52.577	1638
CHI to BOS	60.557	526
CHI to KCY	67.258	214
NOL to CHI	69.959	592
CHI to NYP	72.332	886
CHI to NOL	79.095	539
CHI to WAS	80.285	616
NOL to NYP	80.328	562
CHI to SAS	101.905	587
SEA to LAX	105.785	576
LAX to SEA	110.562	585
CHI to SEA	112.925	572
SPK to PDX	113.36	578
LAX to NOL	133.718	292
LAX to CHI	142.847	589
CHI to LAX	154.379	589
NOL to LAX	175.032	303
SEA to CHI	179.405	579
CHI to EMY	179.9	592
EMY to CHI	183.268	598
PHL to HAR	8.991	1587
GFD to NHV	9.269	1000
SAN to OSD	9.459	435
SPG to GFD	9.697	216
MKE to CHI	10.221	4293
NYP to PHL	10.547	390
SPG to NHV	10.616	2619
PHL to NYP	11.008	111
NYP to RUD	12.621	59
HAR to NYP	13.2